In [12]:
#!python3 -m spacy download en_core_web_sm
#!pip install geopy
import pandas as pd
import numpy as np
import NLP
from dataprep.clean import validate_country
import altair as alt
# instantiate one
import spacy 
nlp = spacy.load('en_core_web_sm')
#import geopy.geocoder
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from functools import partial
from tqdm import tqdm
import folium 
import pandas as pd
tqdm.pandas()
import re

In [22]:
# This code is reading from the original csv file that is 1000 rows long 
df = pd.read_csv('metaverse_tweets.csv')
print(df.shape)


(1000, 13)


## Cleaning Twitter Location Data

Removing #'s from the data

In [14]:
df.user_location.astype(str)
df = df[~df.user_location.str.contains("#", na=False)] 
df.head(20)                                  

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Diep Ngoc,NaN,NaN,2021-08-22 09:53:23+00:00,162,129,1816,False,2022-02-27 23:59:46+00:00,@KingArthurrNFT My choice is #Elemon 😊🤩😍🥰\n#BS...,"['Elemon', 'BSC', 'NFT', 'GameFi', 'Metaverse'...",Twitter for Android,False
1,Duncs,"Melbourne, Australia",NFT's + Crypto + Metaverse + Outdoors + Goodtimes,2012-11-28 01:47:19+00:00,745,1363,2583,False,2022-02-27 23:59:33+00:00,Can't wait to hear this #Bapesclan \n\n#BapesM...,"['Bapesclan', 'BapesMetavestor', 'Metaverse']",Twitter Web App,False
2,Putik Gita,"Banyuwangi, Indonesia",just ordinary woman,2016-04-07 03:31:10+00:00,21,130,65,False,2022-02-27 23:59:26+00:00,@SuperStarPad @binance @Nuls good project.. ho...,NaN,Twitter for Android,False
3,🍑𝗔𝗕𝗗 𝗖𝗫 𝟏𝟎𝟎𝐊🍑,"Dhaka, Bangladesh",🍑🌱🍑𝐅𝐎𝐋𝐋𝐎𝐖 𝐌𝐄 🍑🌱🍑\n𝐈 𝐅𝐎𝐋𝐋𝐎𝐖 𝐁𝐀𝐂𝐊 𝐅𝐀𝐒𝐓🍑𝟏𝟎𝟎%𝐈𝐅𝐁🍑\...,2021-10-15 18:21:33+00:00,2355,3349,8550,False,2022-02-27 23:59:13+00:00,Starts Trading on Uniswap and XT Exchanges Feb...,NaN,Twitter Web App,False
4,Ilyass,NaN,Cool,2021-07-30 15:48:01+00:00,4,130,115,False,2022-02-27 23:59:05+00:00,@hustleofwargame #P2E #GameFi #Metaverse #Game...,"['P2E', 'GameFi', 'Metaverse', 'GameNFTs', 'Bl...",Twitter for Android,False
5,Lyxo🌿🌿,"Uyo,Akwa Ibom",Anatomist||Man utd faithful || Music freak || ...,2019-09-16 00:42:59+00:00,849,94,26422,False,2022-02-27 23:59:04+00:00,@supertobi64 #MiniKishu is the next BIG projec...,"['MiniKishu', 'BSC', 'gaming', 'staking']",Twitter for iPhone,False
6,Diep Ngoc,NaN,NaN,2021-08-22 09:53:23+00:00,162,129,1816,False,2022-02-27 23:58:58+00:00,@aegean2356 My choice is #Elemon 🥰🤩💖💗💞\n#BSC ...,"['Elemon', 'BSC', 'NFT', 'GameFi', 'Metaverse'...",Twitter for Android,False
7,🈹ㄒ山|几🈯️,"Atlanta, GA📍",𝕋𝕚𝕞𝕞𝕚𝕚 T̆̈𝙬𝙞𝙣T̑̈𝙬𝙞𝙣 || This the link: https://...,2013-06-03 18:46:28+00:00,3965,4975,119194,False,2022-02-27 23:58:57+00:00,NFT community!!! 📣\nYa go follow @twin_nfts. W...,NaN,Twitter for iPhone,False
8,Lyxo🌿🌿,"Uyo,Akwa Ibom",Anatomist||Man utd faithful || Music freak || ...,2019-09-16 00:42:59+00:00,849,94,26422,False,2022-02-27 23:58:46+00:00,@PeeGee05 @MatrixETF @DEnergizer645 @anietieet...,"['MiniKishu', 'BSC']",Twitter for iPhone,False
9,Rabby78,NaN,Like comments flow korben plz,2021-08-30 19:02:38+00:00,30,1166,1767,False,2022-02-27 23:58:32+00:00,The project is great and this projector has a ...,NaN,Twitter for Android,False


Removing Na's

In [23]:
df = df[df["user_location"].notna()]

## Getting rid of the emojis in the location

In [17]:
import re 

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F" #emoticons
        u"\U0001F300-\U0001F5FF" #symbols & Pictographs
        u"\U0001F680-\U0001F6FF" #transport & map symbols
        u"\U0001F1E0-\U0001F1FF" #flags IOS
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

df["user_location"] = df["user_location"].map(lambda x: remove_emoji(x))
df.head(20)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
1,Duncs,"Melbourne, Australia",NFT's + Crypto + Metaverse + Outdoors + Goodtimes,2012-11-28 01:47:19+00:00,745,1363,2583,False,2022-02-27 23:59:33+00:00,Can't wait to hear this #Bapesclan \n\n#BapesM...,"['Bapesclan', 'BapesMetavestor', 'Metaverse']",Twitter Web App,False
2,Putik Gita,"Banyuwangi, Indonesia",just ordinary woman,2016-04-07 03:31:10+00:00,21,130,65,False,2022-02-27 23:59:26+00:00,@SuperStarPad @binance @Nuls good project.. ho...,NaN,Twitter for Android,False
3,🍑𝗔𝗕𝗗 𝗖𝗫 𝟏𝟎𝟎𝐊🍑,"Dhaka, Bangladesh",🍑🌱🍑𝐅𝐎𝐋𝐋𝐎𝐖 𝐌𝐄 🍑🌱🍑\n𝐈 𝐅𝐎𝐋𝐋𝐎𝐖 𝐁𝐀𝐂𝐊 𝐅𝐀𝐒𝐓🍑𝟏𝟎𝟎%𝐈𝐅𝐁🍑\...,2021-10-15 18:21:33+00:00,2355,3349,8550,False,2022-02-27 23:59:13+00:00,Starts Trading on Uniswap and XT Exchanges Feb...,NaN,Twitter Web App,False
5,Lyxo🌿🌿,"Uyo,Akwa Ibom",Anatomist||Man utd faithful || Music freak || ...,2019-09-16 00:42:59+00:00,849,94,26422,False,2022-02-27 23:59:04+00:00,@supertobi64 #MiniKishu is the next BIG projec...,"['MiniKishu', 'BSC', 'gaming', 'staking']",Twitter for iPhone,False
7,🈹ㄒ山|几🈯️,"Atlanta, GA",𝕋𝕚𝕞𝕞𝕚𝕚 T̆̈𝙬𝙞𝙣T̑̈𝙬𝙞𝙣 || This the link: https://...,2013-06-03 18:46:28+00:00,3965,4975,119194,False,2022-02-27 23:58:57+00:00,NFT community!!! 📣\nYa go follow @twin_nfts. W...,NaN,Twitter for iPhone,False
8,Lyxo🌿🌿,"Uyo,Akwa Ibom",Anatomist||Man utd faithful || Music freak || ...,2019-09-16 00:42:59+00:00,849,94,26422,False,2022-02-27 23:58:46+00:00,@PeeGee05 @MatrixETF @DEnergizer645 @anietieet...,"['MiniKishu', 'BSC']",Twitter for iPhone,False
10,Duncs,"Melbourne, Australia",NFT's + Crypto + Metaverse + Outdoors + Goodtimes,2012-11-28 01:47:19+00:00,745,1363,2583,False,2022-02-27 23:58:22+00:00,Cant wait for #spacebapes \n\n#bapesclan #Meta...,"['spacebapes', 'bapesclan', 'Metaverse', 'bape...",Twitter Web App,False
15,NFTcrawler.eth,NFTverse,#mww,2009-07-04 13:44:10+00:00,193,97,347,False,2022-02-27 23:56:32+00:00,@whitesandsgame @nftworldsNFT White Sands will...,['Metaverse'],Twitter Web App,False
16,HQ Muhammad Ashikur Rahman,Sreemongal,https://t.co/YXClUUDjtR\n\nBismillahir Rahmani...,2021-04-21 14:29:40+00:00,445,2239,2663,False,2022-02-27 23:56:27+00:00,@TAROVERSEcom I've participated with the rules...,NaN,Twitter Web App,False
17,0x888nft,Radioactive world,0x888nft.eth:\n#WomenOfCrypto-#784 (42/8888)\n...,2022-01-15 00:33:52+00:00,193,35,110,False,2022-02-27 23:56:17+00:00,"In the world of #RadioactiveApes, don't forget...","['RadioactiveApes', 'nfts', 'NFTCommunity', '3...",Twitter for Android,False


Using GeoPy API to get coordinates of locations 

In [18]:
# Ref: https://geopy.readthedocs.io/en/stable/#usage-with-pandas
geolocator = Nominatim(user_agent='my_email@myserver.com')

#Currently Have a weird Rate Limiter error not sure what to do 

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3, max_retries=5)
df["location"] = df["user_location"].progress_apply(geocode, language="en") # Some locations are in hindi, chinese. Language=’en’ returns location in english
df["coordinates"] = df["location"].apply(lambda loc: tuple(loc.point) if loc else None)
df["state"] = df["location"].apply(lambda loc: loc[0].split(",")[0] if loc else None)
df["country"] = df["location"].apply(lambda loc: loc[0].split(",")[-1] if loc else None)


 18%|███████▋                                  | 97/530 [04:45<21:39,  3.00s/it]Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=East+Java%2C+Indonesia&format=json&limit=1&accept-language=en
Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=East+Java%2C+Indonesia&format=json&limit=1&accept-language=en
 28%|███████████▌                             | 150/530 [07:24<19:04,  3.01s/it]Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,location,coordinates,state,country
1,Duncs,"Melbourne, Australia",NFT's + Crypto + Metaverse + Outdoors + Goodtimes,2012-11-28 01:47:19+00:00,745,1363,2583,False,2022-02-27 23:59:33+00:00,Can't wait to hear this #Bapesclan \n\n#BapesM...,"['Bapesclan', 'BapesMetavestor', 'Metaverse']",Twitter Web App,False,"(Melbourne, City of Melbourne, Victoria, Austr...","(-37.8142176, 144.9631608, 0.0)",Melbourne,Australia
2,Putik Gita,"Banyuwangi, Indonesia",just ordinary woman,2016-04-07 03:31:10+00:00,21,130,65,False,2022-02-27 23:59:26+00:00,@SuperStarPad @binance @Nuls good project.. ho...,NaN,Twitter for Android,False,"(Banyuwangi, East Java, Indonesia, (-8.2094973...","(-8.2094973, 114.3737201, 0.0)",Banyuwangi,Indonesia
3,🍑𝗔𝗕𝗗 𝗖𝗫 𝟏𝟎𝟎𝐊🍑,"Dhaka, Bangladesh",🍑🌱🍑𝐅𝐎𝐋𝐋𝐎𝐖 𝐌𝐄 🍑🌱🍑\n𝐈 𝐅𝐎𝐋𝐋𝐎𝐖 𝐁𝐀𝐂𝐊 𝐅𝐀𝐒𝐓🍑𝟏𝟎𝟎%𝐈𝐅𝐁🍑\...,2021-10-15 18:21:33+00:00,2355,3349,8550,False,2022-02-27 23:59:13+00:00,Starts Trading on Uniswap and XT Exchanges Feb...,NaN,Twitter Web App,False,"(Dhaka, Dhaka Metropolitan, Dhaka District, Dh...","(23.7861979, 90.4026151, 0.0)",Dhaka,Bangladesh
5,Lyxo🌿🌿,"Uyo,Akwa Ibom",Anatomist||Man utd faithful || Music freak || ...,2019-09-16 00:42:59+00:00,849,94,26422,False,2022-02-27 23:59:04+00:00,@supertobi64 #MiniKishu is the next BIG projec...,"['MiniKishu', 'BSC', 'gaming', 'staking']",Twitter for iPhone,False,"(Akwa Ibom, Abak Road, Ediene, Abak, Akwa Ibom...","(4.9408638, 7.8412267, 0.0)",Akwa Ibom,Nigeria
7,🈹ㄒ山|几🈯️,"Atlanta, GA",𝕋𝕚𝕞𝕞𝕚𝕚 T̆̈𝙬𝙞𝙣T̑̈𝙬𝙞𝙣 || This the link: https://...,2013-06-03 18:46:28+00:00,3965,4975,119194,False,2022-02-27 23:58:57+00:00,NFT community!!! 📣\nYa go follow @twin_nfts. W...,NaN,Twitter for iPhone,False,"(Atlanta, Fulton County, Georgia, United State...","(33.7489924, -84.3902644, 0.0)",Atlanta,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Asianpett🅰️ 🕊️ ⚪,"Akwa Ibom, Nigeria",graphics designer\n✍️ . Personal Art | Cover A...,2020-06-16 20:09:17+00:00,507,1228,3826,False,2022-02-27 22:08:45+00:00,@AltcoinDailyio I really do believe in @YOMeta...,NaN,Twitter for Android,False,"(Akwa Ibom, Nigeria, (4.9880607, 7.79517827785...","(4.9880607, 7.795178277855076, 0.0)",Akwa Ibom,Nigeria
996,Scorpion Design,UK,"Mathematical Physicist. Digital art, conceptua...",2014-05-10 16:10:39+00:00,7860,7914,9994,False,2022-02-27 22:08:42+00:00,Check out this new digital art that I uploaded...,['art'],Twitter Web App,False,"(United Kingdom, (54.7023545, -3.2765753))","(54.7023545, -3.2765753, 0.0)",United Kingdom,United Kingdom
997,Scorpion Design,UK,"Mathematical Physicist. Digital art, conceptua...",2014-05-10 16:10:39+00:00,7860,7914,9994,False,2022-02-27 22:08:34+00:00,Check out this new digital art that I uploaded...,['art'],Twitter Web App,False,"(United Kingdom, (54.7023545, -3.2765753))","(54.7023545, -3.2765753, 0.0)",United Kingdom,United Kingdom
998,Scorpion Design,UK,"Mathematical Physicist. Digital art, conceptua...",2014-05-10 16:10:39+00:00,7860,7914,9994,False,2022-02-27 22:08:18+00:00,Check out this new digital art that I uploaded...,['art'],Twitter Web App,False,"(United Kingdom, (54.7023545, -3.2765753))","(54.7023545, -3.2765753, 0.0)",United Kingdom,United Kingdom


In [21]:
df.to_csv('metaverse_tweets_final.csv', encoding='utf-8', index = False)

my_map = folium.Map(
    location=[13.133932434766733, 16.103938729508073],
    zoom_start=2)
my_map

In [29]:


df = df[df['coordinates'].notna()]
for index, row in df.iterrows():
    #print(row['coordinates'], row['user_location'])
    lat, lon, _ = row['coordinates']
    folium.Marker(
        location = [lat, lon],
        popup= row['user_location'],
        tooltip = row['user_location'],
    ).add_to(my_map)
    
my_map

KeyError: 'coordinates'